In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install ipympl
!{sys.executable} -m pip install mplcursors
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install flatbuffers
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install numba
!{sys.executable} -m pip install d3blocks

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2f/0e/3b74e8f7c908082793adafb02753477f653ccd7e189f3ba070757d2d0e65/pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 18.2 MB/s eta 0:00:00
  Using cached ipympl-0.9.3-py2.py3-none-any.whl (511 kB)
  Using cached mplcursors-0.5.2-py3-none-any.whl
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/18/44/7e8d208eb59a8224fcc474415104f13be9b378be8da63f76dfde12ec2b44/scipy-1.11.3-cp310-cp310

In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt 
import os
taz_path = os.path.join(os.path.abspath(os.getcwd()),"../../")
print(f"Detected Taz path is {taz_path}") 
sys.path.append(os.path.join(taz_path,"scripts"))
import taz
import fbs
from taz import matrix
from taz import snapshot
import generate_apps_traffic as gen
import fbs.taz.profile
import pandas as pd
import glob
import math
import statistics 

# To play with libraries themselves
import importlib
# importlib.reload(gen)
importlib.reload(taz)
importlib.reload(taz.matrix)
importlib.reload(fbs)
importlib.reload(fbs.taz.profile)
importlib.reload(fbs.taz.profile.Matrix)
importlib.reload(fbs.taz.profile.Property)

def open_single_run_result(run_result_folder, retrieve_mappings, retrieve_snapshots ) :
    abs_folder=os.path.join(taz_path,run_result_folder)
    jobs_file=os.path.join(abs_folder,"jobs_0.csv")
    if not os.path.exists(jobs_file):
        return None
    jobs = pd.read_csv(jobs_file)
    stats_file=os.path.join(abs_folder,"stats_0.csv")
    #print(f"{stats_file=}")
    stats = pd.read_csv(stats_file)
    mappings={}
    if retrieve_mappings:
        for mapping_file in glob.glob(f"{abs_folder}/mapping*"):
            mat= taz.matrix.Matrix()
            mat.read(mapping_file)
            mappings[os.path.basename(mapping_file)]=mat
    
    snapshots={}        
    if retrieve_snapshots:
        for snapshot_file in glob.glob(f"{abs_folder}/snap_*"):
            s=taz.snapshot.Snapshot(snapshot_file)
            snapshots[os.path.basename(snapshot_file)]=s

    completed_jobs=jobs[jobs['endtype']=='COMPLETION']
    n_failed_jobs=len(jobs[jobs['endtype']!='COMPLETION'])
    nominal=sum(completed_jobs['nominal'])
    nominal_node_time=sum(completed_jobs['nominal']*completed_jobs['nnodes']*completed_jobs['nnodes'])
    runtime=sum(completed_jobs['end']-completed_jobs['start'])
    count=completed_jobs.shape[0]
    #print(f"Got {nominal=} {runtime=} {count=}")

    return {'jobs': jobs, 'x': [nominal,runtime,nominal_node_time,count,n_failed_jobs], 'stats':stats, 'mappings':mappings , 'snapshots':snapshots}


Detected Taz path is /home/chaix/deep-sea/taz/experiments/faultfree_mapping/../../


In [3]:
results_base_folder="results"
#results_base_folder="results_faultfree_mapping"
retrieve_mappings=True
retrieve_snapshots=True
results_linear=open_single_run_result(results_base_folder+"/faultfree_mapping_experiment_m0", retrieve_mappings, retrieve_snapshots )
results_uniform=open_single_run_result(results_base_folder+"/faultfree_mapping_experiment_m1", retrieve_mappings, retrieve_snapshots )
results_profile=open_single_run_result(results_base_folder+"/faultfree_mapping_experiment_m2", retrieve_mappings, retrieve_snapshots )

In [4]:
#def show_occupancy( jobs, nnodes, endtime, time_quanta):
jobs=results_profile["jobs"]#[1:10]
nnodes=1024 
endtime=900000
time_quanta=1000
results = results_linear

In [5]:
    
def get_color_string(arr):
    return f"#{arr[0]:02X}{arr[1]:02X}{arr[2]:02X}"

greys = [
    [0xB3,0xB3,0xB3],  
    [0x66,0x66,0x66], 
    [0x1A,0x1A,0x1A]]

#Divergent color map
div_color_map=[
    [0x80,0xB3,0xFF],
    [0xAA,0xDE,0x87],
    [0xFF,0xE6,0x80],
    [0xFF,0xB3,0x80],
    [0xFF,0x80,0x80],
    #second set of colors
    [0x2A,0x7F,0xFF], 
    [0x71,0xC8,0x37],
    [0xFF,0xD4,0x28],
    [0xFF,0x7F,0x2A],
    [0xFF,0x2A,0x2A],
    #third
    [0x00,0x44,0xAA],
    [0x44,0x78,0x21],
    [0xD4,0xAA,0x00],
    [0xD4,0x55,0x00],
    [0xD4,0x00,0x00]]


next_div_color_idx=0

def get_div_color(used_cindices):
    global next_div_color_idx
    ncolors=len(div_color_map)
    assert(len(used_cindices) < ncolors)
    #used_cindices.sort()
    #idx=min([x for x in range(len(div_color_map)) if not x in used_cindices ])
    #print(f"Given used indices {used_cindices}, I selected index {idx}")
    idx=next_div_color_idx
    next_div_color_idx=(idx+1)%ncolors
    return [idx,get_color_string(div_color_map[idx])]
            
def get_seq_color(x,xrange):
    #Sequential color map
    seq_color_map_neg=[
        [0x80,0xB3,0xFF],#->blues
        [0x2A,0x7F,0xFF],#
        [0x00,0x44,0xAA],#
        [0xAA,0xDE,0x87],#-> greens
        [0x71,0xC8,0x37],#
        [0x44,0x78,0x21]]#
    
    seq_color_zero = [0xB3,0xB3,0xB3]
        
    seq_color_map_pos=[
        [0xFF,0xE6,0x80],#-> yellows
        [0xFF,0xD4,0x28],#
        [0xD4,0xAA,0x00],#
        [0xFF,0xB3,0x80],#-> oranges
        [0xFF,0x7F,0x2A],#
        [0xD4,0x55,0x00],#
        [0xFF,0x80,0x80],#-> reds
        [0xFF,0x2A,0x2A],#
        [0xD4,0x00,0x00]]
    
    assert( x >= xrange[0])
    assert( x <= xrange[1])
        
    if x < 0:
        ratio= - x * ( len(seq_color_map_neg) + .49) / xrange[1]
        idx=int(np.round( ratio ) )
        #print(f" neg {ratio=} {idx=}")
        if idx == 0:
            return get_color_string(seq_color_zero)
        return get_color_string(seq_color_map_neg[idx-1])
    
    ratio= x * ( len(seq_color_map_pos) + .49) / xrange[1]
    idx=int(np.round( ratio ) )
    #print(f"pos {ratio=} {idx=}")
    if idx == 0:
        return get_color_string(seq_color_zero)
    return get_color_string(seq_color_map_pos[idx-1])
    
def get_mapping_file(j_id) :
    mapping_filename=f"mapping_{j_id}.tazm"
    if(not mapping_filename in results['mappings']):
        return None
    return results['mappings'][mapping_filename]

def get_mapping_rectangles(j_id, j_start, j_end,next_rect_id):
    first_node=-1
    last_node=-1
    vis_rects=[]
    mp=get_mapping_file(j_id)
    if(not mp):
        print(f"Could not find mapping file for job {j_id}, skipping it")
        return  [next_rect_id,vis_rects]

    for i_arr  in np.nditer(mp.np_array):    
        i=int(i_arr)
        #print(f"{mapping_filename=} {i=}")
        #Stay in the group of nodes
        if first_node >= 0 and last_node + 1 == i:
            last_node += 1
            continue
            
        #Save the past group of nodes
        if first_node >= 0:
            vis_rects.append({'rect_id':next_rect_id, 'x':j_start, 'y': last_node, 'width':j_end-j_start, 'height':last_node-first_node+1, 'job_id':j_id, 'view':'nodes'})
            next_rect_id+=1
            #print(f"{mapping_filename=} {i=} add rect {first_node=} {last_node=}")    

        #Create a new group of nodes
        last_node=first_node=i

    if first_node >= 0:
        vis_rects.append({'rect_id':next_rect_id,'x':j_start, 'y': last_node, 'width':j_end-j_start, 'height':last_node-first_node, 'job_id':j_id, 'view':'nodes'})
        next_rect_id+=1
        #print(f"{mapping_filename=} {i=} add rect (2) {first_node=} {last_node=}")    
    return  [next_rect_id,vis_rects]
    
jobs_sorted=jobs.sort_values(["start","end"])

event_times = list(set( list(jobs_sorted["start"]) +list(jobs_sorted["end"])))
event_times.sort()
n_times=len(event_times)
n_jobs=len(jobs.index)
slowdown_range=[ np.nanmin(jobs_sorted['slowdown']), np.nanmax(jobs_sorted['slowdown'])]
if slowdown_range[0] == slowdown_range[1] :
    slowdown_range[1] = slowdown_range[0] + 1.0

job_stack_offset = 0

job_pack_last_time=0
job_pack_last_update_time=0
job_pack_last_count=0

#rect_id: Unique rectangle identifier
#x: left coordinate of rectangle
#y: upper coordinate of rectanble
#width: width of the rectangle
#height: height of the rectangle
#job_id: the id of the parent job
#view: the view it belongs to, amongst 'job_stack', 'job_pack' or 'nodes' 
vis_rects= []
next_rect_id=0
alive_jobs = []

#id: Id of the job
#desc: a string describing the job
#c_divergent: color for divergent color scheme (1 job ~ 1 color)
#c_slowdown: color for slowdown  color scheme
vis_jobs =[]

for j_index , j in jobs_sorted.iterrows():
    j_id     = j['id']
    j_start  = j["start"]
    j_end    = j["end"]
    j_endtype= j["endtype"]
    j_nnodes = j["nnodes"]
    
    app_name="N.A."
    mapping_file=get_mapping_file(j_id)
    if mapping_file :
        app_name=mapping_file.properties['job_source'][0].split('/')[-2]
    j_desc=f"Job {j_id} ({app_name}) with {j_nnodes} nodes\navail.:{j['availability']} start:{j['start']} end:{j['end']}\ndue to {j['endtype']}"

    #print(f"Processing {j_desc}...")
    

    #handle ended jobs and create job_pack rects
    if j_start == job_pack_last_time:
        job_pack_last_count += j_nnodes
    else:
        ended_jobs = []
        new_alive_jobs = []
        new_n=j_nnodes
        for x in alive_jobs:
            if  x['end'] > j_start:
                new_alive_jobs.append(x)
                new_n += x['nnodes'] 
            else:
                ended_jobs.append(x)
        #Build rectangles for the jobs pack view
        ended_jobs.sort(key=lambda x: x['end'])
        
        #print(f"{new_alive_jobs=} {ended_jobs=}")
        
        t=job_pack_last_update_time
        n=job_pack_last_count
        for x in ended_jobs:
            last_t=t
            t=x['end']
            n-=x['nnodes']
            if x['end'] != last_t:
                if(job_pack_last_count):
                    vis_rects.append({'rect_id':next_rect_id,'x':job_pack_last_update_time, 'y':job_pack_last_count ,
                                 'width':t-job_pack_last_update_time, 'height':job_pack_last_count,
                                 'job_id':-1, 'view':'job_pack'})
                    next_rect_id+=1
                #print(f"Add pack rectangle because job {x['id']} ends at time {x['end']} {job_pack_last_count=}@{job_pack_last_update_time=}")
                job_pack_last_count = n
                job_pack_last_update_time = t
        
        if new_n != n:
            if(job_pack_last_count):
                vis_rects.append({'rect_id':next_rect_id,'x':job_pack_last_update_time, 'y':job_pack_last_count ,
                             'width':j_start-job_pack_last_update_time, 'height':job_pack_last_count,
                             'job_id':-1, 'view':'job_pack'})
                next_rect_id+=1
                #print(f"Add pack rectangle because job {j_id} (and others?) start at time {j_start} {job_pack_last_count=}@{job_pack_last_update_time=}")
            job_pack_last_count = new_n
            job_pack_last_update_time = j_start
            
        alive_jobs=new_alive_jobs
    
    job_pack_last_time=j_start
    
    #Select a color 
    cindex=jc_divergent=jc_slowdown=None
    if j_endtype == "COMPLETION" :
        used_cindices = [ x['cindex'] for x in alive_jobs if x['cindex'] != None ]
        [cindex,jc_divergent] = get_div_color(used_cindices) 
        jc_slowdown = get_seq_color(j["slowdown"],slowdown_range)
    elif j_endtype == "LINK_FAILURE" :
        jc_divergent = jc_slowdown = "url(#linkFailurePattern)"
    elif j_endtype == "HOST_FAILURE" :
        jc_divergent = jc_slowdown = "url(#hostFailurePattern)"
    elif j_endtype == "TIMEOUT" :
        jc_divergent = jc_slowdown = "url(#timeoutPattern)"
    else:
        print(f"Error, I do not know endtype {j_endtype}")
    
    alive_jobs.append({'id':j_id,'end':j_end,'nnodes':j_nnodes, 'cindex':cindex})
        
    #Build rectangle for the jobs stack view
    vis_rects.append({'rect_id':next_rect_id,'x':j_start, 'y': job_stack_offset+j_nnodes, 'width':j_end-j_start, 'height':j_nnodes, 'job_id':j_id, 'view':'job_stack'})
    next_rect_id+=1
    job_stack_offset += j_nnodes
    
    #Build rectangles for the nodes view
    [next_rect_id,new_vis_rects]=get_mapping_rectangles(j_id,j_start,j_end,next_rect_id)    
    vis_rects+=new_vis_rects
    
    #print(f"{j_index=} {n_times=} {len(x)=} {j=}")
    vis_jobs.append({'id':j_id, 'desc': j_desc, 'c_divergent':jc_divergent ,'c_slowdown':jc_slowdown })

    
#Handle the jobs that finish after the last job started
t=job_pack_last_update_time
n=job_pack_last_count
alive_jobs.sort(key=lambda x: x['end'])
for x in alive_jobs:
    last_t=t
    t=x['end']
    n-=x['nnodes']
    if x['end'] != last_t:
        if(job_pack_last_count):
            vis_rects.append({'x':job_pack_last_update_time, 'y':job_pack_last_count ,
                         'width':t-job_pack_last_update_time, 'height':job_pack_last_count,
                         'job_id':-1, 'view':'job_pack'})
        #print(f"Add pack rectangle (2) because job {x['id']} ends at time {x['end']} {job_pack_last_count=}@{job_pack_last_update_time=}")
        job_pack_last_count = n
        job_pack_last_update_time = t

pd.DataFrame(vis_rects).to_csv('rectangles.csv')
pd.DataFrame(vis_jobs).to_csv('jobs.csv') 
# vis_rects

If we get a CORS problem, we need to modify the server configuration:
- jupyter server --generate-config
- Modify the configuration file to get: c.ServerApp.disable_check_xsrf = True
- restart jupyter-lab again

In [1]:
%%javascript

// Based on https://stackoverflow.com/questions/66060012/display-d3-js-visualisation-in-jupyter-lab
var script = document.createElement('script');
script.type = 'text/javascript';
script.src = '//d3js.org/d3.v7.js'
document.head.appendChild(script);
//console.error(window.d3)

//------------------------------------------------------------------
// Configuration of the plot size
//------------------------------------------------------------------

const width = 1000;
const height = 800;
const marginTop = 20;
const marginRight = 20;
const marginBottom = 30;
const marginLeft = 60;

//------------------------------------------------------------------
// Global variables used to draw stuff
//------------------------------------------------------------------

var divWidgets
var divSvg


var zoom_transform
var max_time
var max_y
var rectangles 
var jobs
var view_types = ["job_stack", "job_pack", "nodes"]
var current_view_type = view_types[0]
var color_schemes = ["jobs_divergent", "slowdown"]
var current_color_scheme = color_schemes[0]
var job_selected=-1

divWidgets=d3.select(element).append("div").attr("name","widgets")
divSvg=d3.select(element).append("div").attr("name","svg")

//------------------------------------------------------------------
// The coloring function
//------------------------------------------------------------------

var get_color=function(job_id) {
    if(job_id<0 || (job_selected >= 0 && job_selected != job_id ) )
        return '#B3B3B3' // Grey
    var j=jobs.filter((d) => d.id==job_id)
    // console.log(j)
    if(j.length!=1)
        console.warning("Could not find job "+job_id+"entry")
    var c = j[0].c_slowdown
    if(current_color_scheme=="jobs_divergent")
        c=j[0].c_divergent
    //console.log("Color for job "+job_id+" is "+c+" from entry "+j)
    return c
}
    
//------------------------------------------------------------------
// The main drawing function
//------------------------------------------------------------------

function update_plot() {    
    
    
    if(rectangles == undefined || jobs== undefined) {
        console.log("Waiting for input data...")
        return
    }

    var rectangles_data=rectangles.filter(function(d) {
        return d.view == current_view_type 
    })

    max_time= d3.max(rectangles_data,(d) => d.x+d.width)
    const x = zoom_transform.rescaleX(d3.scaleLinear().range([marginLeft,width - marginRight]).domain([0, max_time]))
    max_y= d3.max(rectangles_data, (d) => d.y)
    const y = zoom_transform.rescaleY(d3.scaleLinear().range([height - marginBottom, marginTop]).domain([0, max_y]))
    //console.log('max_time='+max_time+' max_y='+max_y)
    //console.log('y0='+y(0)+' y16='+y(16))    
    
    var rects = svg.selectAll("rect.z")
        .data(rectangles_data
        .filter(function(d) {
            //Just do not spend time on rectangles that will not be in the plot area
            //Rely on clipping for the esthetics
            return x(d.x+d.width) > marginLeft && x(d.x) < width - marginRight &&
                   y(d.y-d.height) > marginTop &&  y(d.y) < height - marginBottom 
        })
              , 
              function(d){ return d.rect_id})

    function get_x(d) { return Math.max(marginLeft,x(d.x)); }
    function get_y(d) { return Math.max(marginTop,y(d.y)); }
    function get_width(d) { return Math.min(x(d.x + d.width), width-marginRight)- get_x(d) }
    function get_height(d) { return Math.min(y(d.y - d.height), height-marginBottom)- get_y(d) }
    
    rects.exit().remove()
    rects.style("fill", (d) =>  get_color(d.job_id))
        //.attr("transform", zoom_transform)
          .attr("x",get_x).attr("y",get_y)
          .attr("height", get_height).attr("width", get_width)
          //.attr("clip-path","url(#background)")
    rects.enter()
        .append("rect")
          .style("stroke", "gray")
          .style("fill", (d) =>  get_color(d.job_id))
          .attr("x",get_x).attr("y",get_y).attr("height", get_height).attr("width", get_width)
          .attr("class","z")
          //.attr("clip-path","url(#background)")
          .on('mouseover', (event, d) => {  tooltip.transition().duration(200).style("opacity", .9);      
            tooltip.text(get_job_description(event,d))//.attr("x",event.clientX).attr("y", event.clientY);
          })
          .on('mouseout', d => {
            tooltip.transition() .duration(500) .style("opacity", 0);
          })
          .on('click', (event, d) => {  
              if(job_selected>=0)
                  job_selected=-1
              else
                  job_selected=d.job_id
              update_plot()
          })
          //.on('dblclick', (event, d) => { svg.call(zoom).call(zoom.transform, d3.zoomIdentity)}) 

    
    //var y2=zoom_transform.rescaleY(y)
    gy.attr("transform", `translate(${marginLeft},0)`)
        .call(d3.axisLeft(y)).transition().duration(500);

    gx.attr("transform", `translate(0,${height - marginBottom})`)
        .call(d3.axisBottom(x)).transition().duration(500);
}

//------------------------------------------------------------------
// Radio button to select the view type
//------------------------------------------------------------------

divWidgets.append("text").text("\u00A0View type:\u00A0")
var radio_view_type = divWidgets.selectAll("span")
                     .data(view_types)
                    .enter().append("span")
            .attr("name", "view_type")
radio_view_type 
    .append("input")
    .attr("type", "radio")
    .attr("name", "mode")
    .attr("value", function(d) {return d;})
    .property("checked", function(d) {
            //console.log("view type "+d+" / "+ current_view_type+"=>"+(d==current_view_type));
            return d==current_view_type;
    })
    .on("change", function(d){
        current_view_type=d.srcElement.value
        console.log("View type changed to "+d.srcElement.value);
        update_plot()
    })
radio_view_type
    .append("label")
    .text(function(d) {return d;})

//------------------------------------------------------------------
// Radio button to select the color scheme
//------------------------------------------------------------------

divWidgets.append("text").text("\u00A0\u00A0Color schemes:\u00A0")
var radio_color_schemes = divWidgets.selectAll("span#color_scheme")
                     .data(color_schemes)
                    .enter().append("span");
radio_color_schemes 
    .append("input")
    .attr("type", "radio")
    .attr("name", "color_scheme")
    .attr("value", function(d) {return d;})
    .property("checked", function(d) {
        //console.log("color scheme d"+d+" / "+ current_color_scheme+"=>"+(d==current_color_scheme));
        return d==current_color_scheme;
    })
    .on("change", function(d){
        current_color_scheme=d.srcElement.value
        console.log("Color scheme changed to "+d.srcElement.value);
        update_plot()
    })
radio_color_schemes
    .append("label")
    .text(function(d) {return d;})

//------------------------------------------------------------------
// Tooltip
//------------------------------------------------------------------

divWidgets.append("text").text("\u00A0\u00A0Description:\u00A0")
var tooltip = divWidgets.append("text").attr("name","tooltip")

var get_job_description=function(event,d) {
    if(d.job_id<0)
        return ''
    var j=jobs.filter((d2) => d2.id==d.job_id)
    if(j.length!=1)
        console.warning("Could not find job "+d.job_id+" entry")
    return j[0].desc
    
    // return '@'+event.clientX+','+event.clientY+' Job '+d.job_id
}

//------------------------------------------------------------------
// Declare the SVG placeholder
//------------------------------------------------------------------

var svg = divSvg
 .append("svg")
 .attr("width", width)
 .attr("height", height)
 .attr("viewBox", [0, 0, width, height])
 .attr("style", "max-width: 100%; height: auto;")

// Define the failure patterns
var defs=svg.append('defs')
//Link failures
defs.append('pattern')
        .attr('id', 'linkFailurePattern')
        .attr('patternUnits', 'userSpaceOnUse')
        .attr('width', 4)
        .attr('height', 4)
    .append('path')
        .attr('d', 'M-1,1 l2,-2 M0,4 l4,-4 M3,5 l2,-2')
        .attr('stroke', '#FF0000')
        .attr('stroke-width', 1);
//Node failures
defs.append('pattern')
        .attr('id', 'nodeFailurePattern')
        .attr('patternUnits', 'userSpaceOnUse')
        .attr('width', 4)
        .attr('height', 4)
    .append('path')
        .attr('d', 'M-1,1 l2,-2 M0,4 l4,-4 M3,5 l2,-2')
        .attr('stroke', '#00FF00')
        .attr('stroke-width', 1);
//Timeout failures
defs.append('pattern')
        .attr('id', 'timeoutPattern')
        .attr('patternUnits', 'userSpaceOnUse')
        .attr('width', 4)
        .attr('height', 4)
    .append('path')
        .attr('d', 'M-1,1 l2,-2 M0,4 l4,-4 M3,5 l2,-2')
        .attr('stroke', '#0000FF')
        .attr('stroke-width', 1);
//A clip path for rectangles
//defs.append("clipPath")
//    .attr("id","clip")
var bbox = element.getBoundingClientRect()
//console.log('bbox='+bbox)
//console.log('bbox_x='+bbox.x)


const gx = svg.append("g")
const gy = svg.append("g")
const gGrid = svg.append("g");

//------------------------------------------------------------------
// Zooming glue logic
//------------------------------------------------------------------

function zoomed(event) {
    console.log("Received zoom event")
    console.log(event.transform)
    zoom_transform=event.transform
  //  const zx = event.transform.rescaleX(x).interpolate(d3.interpolateRound);
  //  const zy = event.transform.rescaleY(y).interpolate(d3.interpolateRound);
  //  rects.attr("transform", transform).attr("stroke-width", 5 / transform.k);
   // gx.call(xAxis, zx);
   // gy.call(yAxis, zy);
   // gGrid.call(grid, zx, zy);
    update_plot()
}

    
const zoom = d3.zoom()
      .scaleExtent([1, 1000])
      .on("zoom",zoomed)
      .constrain(
       function(transform, extent, translateExtent) {
            var dx0 =  transform.applyX(0),
              dx1 =  max_time == undefined ? 0 : transform.applyX(max_time),
              dy0 = transform.applyY(0),
              dy1 =  max_y == undefined ? 0 : transform.invertY(max_y)
          console.log("transform="+transform+" dx1="+ dx0+ " dy1="+dy0) // " extent="+extent+" translateExtent="+translateExtent)
          var dx=  dx0 < 0 ?   0  : -dx0
             // dx=  dx1 < 0 ? -dx1 :  dx
          var dy= 0 // dy0 > 0 ?   0  : -dy0
            //  dy=  dy1 > 0 ? -dy1 :  dy 
          return transform .translate(dx,dy)
        }   
      );



svg.call(zoom).call(zoom.transform, d3.zoomIdentity);
//svg .append("rect")
//    .attr("width",width-marginLeft-marginRight)
//    .attr("height",height-marginTop-marginBottom)
//    .attr("x",marginLeft)
//    .attr("y",marginTop)
//    .attr("fill","yellow")
//    .attr("name","background");

//------------------------------------------------------------------
// Load input data...
//------------------------------------------------------------------

d3.csv('http://localhost:8888/files/rectangles.csv',d3.autoType).then( function(data) {
    //console.log(data)
    rectangles=data
    update_plot()
})

d3.csv('http://localhost:8888/files/jobs.csv',d3.autoType).then( function(data) {
    //console.log(data)
    jobs=data
    update_plot()
})

<IPython.core.display.Javascript object>

In [9]:
results['mappings']

{'mapping_121.tazm': <taz.matrix.Matrix at 0x7fb1828c5b10>,
 'mapping_76.tazm': <taz.matrix.Matrix at 0x7fb1828c6d10>,
 'mapping_264.tazm': <taz.matrix.Matrix at 0x7fb1807344c0>,
 'mapping_183.tazm': <taz.matrix.Matrix at 0x7fb180734700>,
 'mapping_47.tazm': <taz.matrix.Matrix at 0x7fb180735300>,
 'mapping_255.tazm': <taz.matrix.Matrix at 0x7fb1828eda20>,
 'mapping_110.tazm': <taz.matrix.Matrix at 0x7fb1828ee530>,
 'mapping_237.tazm': <taz.matrix.Matrix at 0x7fb1828ed600>,
 'mapping_25.tazm': <taz.matrix.Matrix at 0x7fb1828ed030>,
 'mapping_172.tazm': <taz.matrix.Matrix at 0x7fb1828ec9a0>,
 'mapping_87.tazm': <taz.matrix.Matrix at 0x7fb1828edfc0>,
 'mapping_139.tazm': <taz.matrix.Matrix at 0x7fb1828ed990>,
 'mapping_108.tazm': <taz.matrix.Matrix at 0x7fb1828ede10>,
 'mapping_143.tazm': <taz.matrix.Matrix at 0x7fb1828edde0>,
 'mapping_206.tazm': <taz.matrix.Matrix at 0x7fb1828eddb0>,
 'mapping_14.tazm': <taz.matrix.Matrix at 0x7fb1828ee110>,
 'mapping_251.tazm': <taz.matrix.Matrix at 0x

In [ ]:
# https://stackoverflow.com/questions/66060012/display-d3-js-visualisation-in-jupyter-lab
from IPython.display import Javascript
svg_script = '''
var svg = d3.select(element)
    .append("svg")
    .attr("width", 300)
    .attr("height", 300);    

svg.append("circle")
    .style("stroke", "gray")
    .style("fill", "cyan")
    .attr("r", 130)
    .attr("cx", 150)
    .attr("cy", 150)
    .transition()
        .delay(100)
        .duration(10000)  
        .attr("r", 10)
        .attr("cx", 150)
        .style("fill", "blue"); 
'''

Javascript(svg_script)